In [1]:
#import dependencies
from PIL import Image, ImageDraw
import numpy as np
import os
from Stain_Normalization import stainNorm_Reinhard, stain_utils
import matplotlib.pyplot as plt

#set size of image 
height = 512
width = height

In [2]:
def transform_colour(directory):
    #iterates directory
    for filename in os.listdir(directory):
        im = Image.open(os.path.join(directory, filename))
        orig = im.copy()
        datas = im.getdata()

        #find whitespace corner and turn background pixels magenta (255,0,255)
        #top-left
        if (datas[0][0] > 200 and datas[0][1] > 200 and datas[0][2] > 200):
            ImageDraw.floodfill(im,xy=(0,0),value=(0,0,0),thresh=50)

        #bottom-right    
        if (datas[height*width-1][0] > 200 and datas[height*width-1][1] > 200):
            ImageDraw.floodfill(im,xy=(height-1,width-1),value=(0,0,0),thresh=50)

        #top-right
        if (datas[height*-1+1][0] > 200): #and datas[height*-1][1] > 200 and datas[height*-1][2] > 200 ):
            ImageDraw.floodfill(im,xy=(height-1,0),value=(0,0,0),thresh=50)

        #Bottom-left    
        if (datas[height*width-width+1][0] > 200 and datas[height*width-width+1][1] > 200):
            ImageDraw.floodfill(im,xy=(0,width-1),value=(0,0,0),thresh=50)

        #Save files in transform magenta
        im.save('transform black/' + filename)

def transform_transparent():
    #itterates directory
    for filename in os.listdir('transform black/'):
        img = Image.open(os.path.join('transform black/', filename))
        img = img.convert("RGBA")
        datas = img.getdata()
        newData = []
        
        #For every pixel, if magenta, turn transparent
        for item in datas:
            if item[0] == 0 and item[1] == 0 and item[2] == 0:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
                
        #add and save to new folder
        img.putdata(newData)
        img.save("transform transparent/" + filename)

def transform_complete():
    #Amalgamates blackspace conversion and normalised image. Will check if pixel is black and if so will add a transparent pixel. 
    #If it is not black it will add the pixel from the normalised image 
    for filename in os.listdir('transform black/'):
            black_img = Image.open(os.path.join('transform black/', filename))
            norm_img = Image.open(os.path.join('transform normalised/', filename))

            black_img = black_img.convert("RGBA")
            norm_img = norm_img.convert("RGBA")
            datas = black_img.getdata()
            norm_datas= norm_img.getdata()
            newData = []

            i = 0
            #For every pixel, if black, turn transparent
            for item in datas:
                i = i + 1
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(norm_datas[i-1])

            #add and save to new folder
            black_img.putdata(newData)
            black_img.save("transform complete/" + filename)

def normalise(directory):
    for filename in os.listdir(directory):  
        path = os.path.join(directory, filename)
        image=stain_utils.read_image(path)
        im = Image.fromarray(n.transform(image))
        im.save('transform normalised/' + filename)
        
def clean_dirs():
    for filename in os.listdir(directory):  
        b_path = os.path.join('transform black/', filename)
        n_path = os.path.join('transform normalised/', filename)
        t_path = os.path.join('test/', filename)
        os.remove(b_path)
        os.remove(n_path)
        os.remove(t_path)

In [14]:
BASE_IMAGE = stain_utils.read_image('H&E.png')
n=stainNorm_Reinhard.Normalizer()
n.fit(BASE_IMAGE)

directory = 'test/'
normalise(directory)
transform_colour(directory)
#transform_transparent()
transform_complete()



In [ ]:
clean_dirs()